<a href="https://colab.research.google.com/github/aIsHdz/Diplomado/blob/main/Notas%20/%20Apuntes_pel%C3%ADculas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/movie_review.csv")
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\""The Classic War of the Worlds\"" by Timothy H..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
df = pd.read_csv('/home/ajaimes/Downloads/datos_reviews.csv')
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df['sentiment'], test_size=0.2, random_state=7)
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_pred)
print(f"Naive Bayes Accuracy: {nb_accuracy:.2f}")
nb_report = classification_report(y_test, nb_pred)
print("Naive Bayes Classification Report:")
print(nb_report)
import plotly.express as px
def plot_confusion_matrix(cm, labels):
    fig_cm = px.imshow(cm, labels=dict(x="Predicted", y="Actual", color="Count"),
                       x=labels, y=labels, color_continuous_scale='Viridis', text_auto = True,
                       title="Confusion Matrix")
    fig_cm.update_layout(coloraxis_showscale=False)
    fig_cm.show()
nb_cm = confusion_matrix(y_test, nb_pred)
plot_confusion_matrix(nb_cm, ['Bueno', 'Malo'])

In [ ]:
library(data.table)

dt <- fread("~/Downloads/movie_review.csv")
dim(dt)
str(dt)
dt[, id := NULL]

library(stringi)
library(xml2)

EscapeHTML <- function(string) {
  string <- stri_replace_all_regex(string, "<.*?>", " ")
  xml_text(read_html(paste0("<x>", string, "</x>")))
}

CleanString <- function(string, regex = "[^a-z]+") {
  string <- as.character(string)
  # string <- stri_trans_general(string, "Latin-ASCII")
  string <- stri_trans_tolower(string)
  string <- stri_replace_all_fixed(string, ".", " ")
  string <- stri_trim_both(string)
  string[string == ""] <- NA_character_
  string
}

dt[, review := EscapeHTML(review), by = .(seq_len(nrow(dt)))]
dt[, review := CleanString(review)]

dict <- table(unlist(strsplit(dt$review, " ", fixed = TRUE)))
dict <- sort(dict, decreasing = TRUE)
head(dict)

dict <- dict[!names(dict) %in% stopwords::data_stopwords_nltk$en]
length(dict)
dict <- dict[dict > 1]
length(dict)

lista <- vector("list", nrow(dt))

for (i in seq_len(nrow(dt))) {
  lista[[i]] <- as.integer(names(dict) %in% unlist(strsplit(dt[i, review], " ", fixed = TRUE)))
}

X <- do.call(rbind, lista)
dim(X)

fwrite(cbind(X, sentiment = dt$sentiment), "~/Downloads/datos_reviews.csv")

set.seed(1)
index <- sample.int(nrow(X), 0.8 * nrow(X))

X_train <- X[index, ]
y_train <- dt[index, sentiment]
X_test <- X[-index, ]
y_test <- dt[-index, sentiment]